# Chatbot with Memory

Let's add some history to the interaction and build a chatbot. Unlike many people think. LLMs are fixed in their state. They are trained until a certain cutoff date and do not know anything after that point unless you feed them current information. That is also why LLMs do not remember anything about you or the prompts you send to the model. If the model seems to remember you and what you said it is always because the application you are using (e.g. ChatPGT or the chat function in SAP AI Launchpad) is sending the chat history to the model to provide the conversation history to the model as context.

Below you can find a simple implementation of a chatbot with memory.

The code in this exercise is based on the [help documentation](https://help.sap.com/doc/generative-ai-hub-sdk/CLOUD/en-US/_reference/orchestration-service.html) of the Generative AI Hub Python SDK.

In [2]:
# import init_env
# import variables
from typing import List

# init_env.set_environment_variables()

### Import the packages you want to use

In [3]:
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import Message, SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.service import OrchestrationService

In [ ]:
import json
import os
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
# Inline credentials
with open('creds.json') as f:
    credCF = json.load(f)
 
# Set environment variables
def set_environment_vars(credCF):
    env_vars = {
        'AICORE_AUTH_URL': credCF['url'] + '/oauth/token',
        'AICORE_CLIENT_ID': credCF['clientid'],
        'AICORE_CLIENT_SECRET': credCF['clientsecret'],
        'AICORE_BASE_URL': credCF["serviceurls"]["AI_API_URL"] + "/v2",
        'AICORE_RESOURCE_GROUP': "grounding"
    }
 
    for key, value in env_vars.items():
        os.environ[key] = value
        print(value)
 
# Create AI Core client instance
def create_ai_core_client(credCF):
    set_environment_vars(credCF)  # Ensure environment variables are set
    return AICoreV2Client(
        base_url=os.environ['AICORE_BASE_URL'],
        auth_url=os.environ['AICORE_AUTH_URL'],
        client_id=os.environ['AICORE_CLIENT_ID'],
        client_secret=os.environ['AICORE_CLIENT_SECRET'],
        resource_group=os.environ['AICORE_RESOURCE_GROUP']
    )
 
ai_core_client = create_ai_core_client(credCF)

https://anuragv2-39wjy902.authentication.eu10.hana.ondemand.com/oauth/token
sb-adcd4907-f1a7-462d-ba8e-646390ee4185!b398425|aicore!b540
107d3e9b-9a41-4b30-9b24-a091a45956cd$VmYxjzammFm50xkj1O37HmzgX3maoNrwfrlm99qUhi0=
https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2
llm-deployed


In [ ]:
AICORE_ORCHESTRATION_DEPLOYMENT_URL = "https://api.ai.prodeuonly.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d2d982db268a416f"

### Create the chatbot class

In [6]:
class ChatBot:
    def __init__(self, orchestration_service: OrchestrationService):
        self.service = orchestration_service
        self.config = OrchestrationConfig(
            template=Template(
                messages=[
                    SystemMessage("You are a helpful chatbot assistant."),
                    UserMessage("{{?user_query}}"),
                ],
            ),
            # TODO add a model name here, e.g. gemini-1.5-flash
            llm=LLM(name="gpt-4o"),
        )
        self.history: List[Message] = []

    def chat(self, user_input):
        response = self.service.run(
            config=self.config,
            template_values=[
                TemplateValue(name="user_query", value=user_input),
            ],
            history=self.history,
        )

        message = response.orchestration_result.choices[0].message

        self.history = response.module_results.templating
        self.history.append(message)

        return message.content
    
    def reset(self):
        self.history = []

service = OrchestrationService(api_url=AICORE_ORCHESTRATION_DEPLOYMENT_URL)
bot = ChatBot(orchestration_service=service)

In [7]:
bot.chat("Hello, are you there?")

"Hello! Yes, I'm here. How can I assist you today?"

In [8]:
bot.chat("Do you like CodeJams?")

"While I don't have personal preferences or feelings, I think CodeJams are great events! They offer excellent opportunities for programmers to challenge themselves, learn new skills, and showcase their problem-solving abilities. If you have any questions about participating in a CodeJam, I'd be happy to help!"

In [9]:
bot.chat("Can you remember what we first talked about?")

"I don't have the ability to remember past interactions or conversations. Each session is independent, so I can only reference what we've talked about within this current session. If there's something you'd like to discuss again or if you have any new questions, feel free to let me know!"

In [10]:
bot.history

[Message(role=<Role.SYSTEM: 'system'>, content='You are a helpful chatbot assistant.'),
 Message(role=<Role.USER: 'user'>, content='Hello, are you there?'),
 Message(role=<Role.ASSISTANT: 'assistant'>, content="Hello! Yes, I'm here. How can I assist you today?"),
 Message(role=<Role.SYSTEM: 'system'>, content='You are a helpful chatbot assistant.'),
 Message(role=<Role.USER: 'user'>, content='Do you like CodeJams?'),
 Message(role=<Role.ASSISTANT: 'assistant'>, content="While I don't have personal preferences or feelings, I think CodeJams are great events! They offer excellent opportunities for programmers to challenge themselves, learn new skills, and showcase their problem-solving abilities. If you have any questions about participating in a CodeJam, I'd be happy to help!"),
 Message(role=<Role.SYSTEM: 'system'>, content='You are a helpful chatbot assistant.'),
 Message(role=<Role.USER: 'user'>, content='Can you remember what we first talked about?'),
 Message(role=<Role.ASSISTANT: '

And to prove to you that the model does indeed not remember you, let's delete the history and try again :)

In [11]:
bot.reset()
bot.chat("Can you remember what we first talked about?")

"I'm afraid I can't remember past interactions or conversations, as I don't have the ability to store personal data or memories. Nonetheless, I'm here to assist you with any questions or information you might need. How can I help you today?"

[Next exercise](11-your-chatbot.ipynb)